In [256]:
from facenet_pytorch import InceptionResnetV1, training, fixed_image_standardization
from torch.utils.tensorboard import SummaryWriter
import torch
import torchvision
import pandas as pd
import numpy as np
import os
from PIL import Image


In [257]:
batch_size = 32
epochs = 16
workers = 0 if os.name == 'nt' else 8
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


# Load Data

In [258]:
data = pd.read_pickle('./data.pkl')
data['race'] = np.where(
   (data['race'] != 4) & (data['race'] != 1) , 0.0, data['race']
   )
data['race'] = np.where(
    data['race'] == 4, 2.0, data['race']
)
print(data['race'].value_counts())
data.head()

2.0    2726
0.0     443
1.0     318
Name: race, dtype: int64


,id,race,img_path,absolute_img_path,cropped_path
0,12488.0,2.0,profile pics/60147.jpeg,./data/profile pics/60147.jpeg,./data/cropped/60147.jpeg
1,719703.0,2.0,profile pics/60148.jpeg,./data/profile pics/60148.jpeg,./data/cropped/60148.jpeg
2,722153.0,0.0,profile pics/60149.jpeg,./data/profile pics/60149.jpeg,./data/cropped/60149.jpeg
3,749003.0,0.0,profile pics/60150.jpeg,./data/profile pics/60150.jpeg,./data/cropped/60150.jpeg
5,811618.0,0.0,profile pics/60152.jpeg,./data/profile pics/60152.jpeg,./data/cropped/60152.jpeg


In [259]:
def load_images(input):
    if os.path.exists(input):
        tmp = Image.open(input)
        test = tmp.getbands()
        keep = tmp.copy()
        return keep
    else: return pd.NA

In [260]:
data['face'] = data['cropped_path'].apply(load_images)
data = data.dropna()
print(data.shape)
data.head()

(2477, 6)


,id,race,img_path,absolute_img_path,cropped_path,face
5,811618.0,0.0,profile pics/60152.jpeg,./data/profile pics/60152.jpeg,./data/cropped/60152.jpeg,<PIL.Image.Image image mode=RGB size=160x160 a...
7,865071.0,2.0,profile pics/60154.jpeg,./data/profile pics/60154.jpeg,./data/cropped/60154.jpeg,<PIL.Image.Image image mode=RGB size=160x160 a...
8,988211.0,2.0,profile pics/60155.jpeg,./data/profile pics/60155.jpeg,./data/cropped/60155.jpeg,<PIL.Image.Image image mode=RGB size=160x160 a...
9,1025311.0,2.0,profile pics/60156.jpeg,./data/profile pics/60156.jpeg,./data/cropped/60156.jpeg,<PIL.Image.Image image mode=RGB size=160x160 a...
10,1143891.0,0.0,profile pics/60157.jpeg,./data/profile pics/60157.jpeg,./data/cropped/60157.jpeg,<PIL.Image.Image image mode=RGB size=160x160 a...


# Build dataset and dataloader

In [261]:
tmp = data[['face','race']].reset_index(drop=True)
x = tmp['face'].tolist()
y = tmp['race'].tolist()

In [262]:
class Face_Race_Dataset(torch.utils.data.Dataset):
    def __init__(self, x,y):
        self.x = x
        self.y = y
        self.transform = torchvision.transforms.Compose(
                            [
                            np.float32,
                            torchvision.transforms.ToTensor(),
                            fixed_image_standardization])

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        return self.transform(self.x[index]),torch.tensor(self.y[index],dtype=torch.long)

In [263]:
dataset = Face_Race_Dataset(x,y)

In [264]:
#all_data = torch.utils.data.DataLoader(dataset,batch_size=10,num_workers=workers,shuffle=True)

In [265]:
train_size = int(0.9 * len(dataset))
valid_size = (len(dataset) - train_size)
train_set,valid_size = torch.utils.data.random_split(dataset,[train_size,valid_size])

# train_size = int(0.8 * len(dataset))
# valid_size = (len(dataset) - train_size)
# train_set,valid_size = torch.utils.data.random_split(dataset,[train_size,valid_size])

# valid_size = int(valid_size / 2)
# test_size = valid_size
# valid_set,test_set = torch.utils.data.random_split(valid_size,[valid_size,test_size])

In [266]:
train_loader = torch.utils.data.DataLoader(train_set,batch_size=32,num_workers=workers,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set,batch_size=32,num_workers=workers,shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_set,batch_size=32,num_workers=workers,shuffle=True)
test = next(iter(valid_loader))

C:\Users\zhouy\AppData\Local\Temp/ipykernel_20240/2642461197.py:15: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return self.transform(self.x[index]),torch.tensor(self.y[index],dtype=torch.long)


[tensor([[[[-0.2773, -0.2773, -0.2773,  ...,  0.2930,  0.2930,  0.2930],
           [-0.2773, -0.2773, -0.2773,  ...,  0.3086,  0.3086,  0.3086],
           [-0.2773, -0.2773, -0.2773,  ...,  0.2930,  0.3008,  0.3008],
           ...,
           [ 0.1914,  0.1836,  0.1914,  ...,  0.0977,  0.0977,  0.0977],
           [ 0.1914,  0.1914,  0.1914,  ...,  0.0898,  0.0977,  0.0977],
           [ 0.1914,  0.1914,  0.1992,  ...,  0.0977,  0.0977,  0.0977]],
 
          [[-0.2930, -0.2930, -0.2930,  ...,  0.3086,  0.3086,  0.3086],
           [-0.2930, -0.2930, -0.2930,  ...,  0.3242,  0.3242,  0.3242],
           [-0.2930, -0.2930, -0.2930,  ...,  0.2930,  0.3008,  0.3008],
           ...,
           [-0.0273, -0.0352, -0.0430,  ...,  0.0820,  0.0820,  0.0820],
           [-0.0273, -0.0273, -0.0430,  ...,  0.0820,  0.0898,  0.0898],
           [-0.0273, -0.0273, -0.0352,  ...,  0.0898,  0.0898,  0.0898]],
 
          [[-0.1211, -0.1211, -0.1211,  ...,  0.4023,  0.4023,  0.4023],
           [-

# Define Inception Resnet V1 module

In [267]:
resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=3
).to(device)

# Define optimizer, scheduler, loss, evaluation functions

In [268]:
optimizer = torch.optim.AdamW(resnet.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [5, 10])

loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

# Train model

In [269]:
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, valid_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, valid_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

writer.close()



Initial
----------
Valid |     2/8    | loss:    1.2173 | fps:  329.4587 | acc:    0.3125   

C:\Users\zhouy\AppData\Local\Temp/ipykernel_20240/2642461197.py:15: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return self.transform(self.x[index]),torch.tensor(self.y[index],dtype=torch.long)


Valid |     8/8    | loss:    1.2080 | fps:  396.4773 | acc:    0.2891   

Epoch 1/16
----------
Train |    70/70   | loss:    1.0089 | fps:  134.2928 | acc:    0.6504   
Valid |     8/8    | loss:    0.6693 | fps:  581.0668 | acc:    0.8516   

Epoch 2/16
----------
Train |    70/70   | loss:    0.6908 | fps:  135.4736 | acc:    0.7823   
Valid |     8/8    | loss:    0.5264 | fps:  562.1762 | acc:    0.8555   

Epoch 3/16
----------
Train |    70/70   | loss:    0.6704 | fps:  137.0727 | acc:    0.7857   
Valid |     8/8    | loss:    0.4974 | fps:  569.3310 | acc:    0.8555   

Epoch 4/16
----------
Train |    70/70   | loss:    0.6926 | fps:  137.9244 | acc:    0.7834   
Valid |     8/8    | loss:    0.8425 | fps:  575.3994 | acc:    0.8464   

Epoch 5/16
----------
Train |    70/70   | loss:    0.6649 | fps:  136.8328 | acc:    0.7868   
Valid |     8/8    | loss:    0.4992 | fps:  588.2426 | acc:    0.8529   

Epoch 6/16
----------
Train |    70/70   | loss:    0.6466 | fps:  131

In [270]:
torch.save(resnet.state_dict(),'dataset1_pic_race_model.pt')